In [1]:
import time
# Save timestamp
start = time.time()
import os
import pandas as pd
import numpy as np
import tensorflow as tf
# import matplotlib.pyplot as plt
import json
from datetime import datetime

In [2]:
print(tf.version.VERSION)

2.18.0-dev20240611


### Specify here the NN topology and the dataframe to be loaded

In [3]:
#specify the NN topology here:
#input_layer_nodes = number_of_attributes
number_of_attributes = 2
hidden_layer_nodes = 2
output_layer_nodes = 2


use_case = "netml-iot"
dataset_folder = f"../datasets/nprint-raw/{use_case}/"
features_rankings = f"../datasets/nprint-raw/{use_case}/feature-importance.csv"



###Define the model

In [4]:
BATCH_SIZE = 512
EPOCHS = 500

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

###Import the data

In [5]:
df_train = pd.read_csv(f'{dataset_folder}X.csv')
df_test = pd.read_csv(f'{dataset_folder}X_val.csv')
label_train = pd.read_csv(f'{dataset_folder}y.csv')
label_test = pd.read_csv(f'{dataset_folder}y_val.csv')

df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)

# display(df_train)
# display(df_test)
# display(label_train)
display(label_test)

,label
0,2
1,1
2,2
3,2
4,2
5,2
6,2
7,2
8,1
9,1


In [6]:
#select a sub-dataframe with only the top attrs rate by the AUTOGLUON feature importance algorithm
#load the file with the rankings
features_rankings_df = pd.read_csv(features_rankings)
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

#make a list with only the best features
feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list = feature_list[:number_of_attributes]
# print(feature_list)

#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list]
df_test = df_test[feature_list]
display(df_train)
display(df_test)

,pkt_5_tcp_opt_64,pkt_9_tcp_opt_64,pkt_5_tcp_opt_65
0,1,0,0
1,0,1,0
2,-1,-1,-1
3,1,0,0
4,0,0,0
...,...,...,...
232,0,0,0
233,-1,-1,-1
234,0,0,1
235,-1,1,-1


,pkt_5_tcp_opt_64,pkt_9_tcp_opt_64,pkt_5_tcp_opt_65
0,1,1,0
1,-1,-1,-1
2,-1,-1,-1
3,1,-1,1
4,0,0,0
5,1,0,0
6,0,0,0
7,1,-1,1
8,0,-1,0
9,0,0,0


In [7]:
#ONLY FOR VIDEO STREAM USE CASE
#RTS values conversion for video-streaming use-case
# INTEGER = 8

# mini = []
# maxi = []
# column_list = ["pkt_1_rts", "pkt_2_rts", "pkt_3_rts", "pkt_4_rts", "pkt_5_rts", "pkt_6_rts", "pkt_7_rts", "pkt_8_rts"]
# for column_name in column_list:
#     mini.append(df_test[column_name].min())
#     print(mini)    
#     maxi.append(df_test[column_name].max())
#     print(maxi)

# maximo = max(maxi)
# minimo = min(mini)
# print(maximo)


# display(df_test)
# for column_name in column_list:
#     aux = []
#     for x in df_test[column_name]:
#         normalized_value = (x - minimo) / (maximo - minimo)
#         scaled_value = normalized_value * (2**INTEGER - 1)
#         aux.append(scaled_value)
#     df_test[column_name]=aux    
# display(df_test)

# mini = []
# maxi = []
# for column_name in column_list:
#     mini.append(df_train[column_name].min())
#     print(mini)    
#     maxi.append(df_train[column_name].max())
#     print(maxi)

# maximo = max(maxi)
# minimo = min(mini)
# print(maximo)


# display(df_train)
# for column_name in column_list:
#     aux = []
#     for x in df_train[column_name]:
#         normalized_value = (x - minimo) / (maximo - minimo)
#         scaled_value = normalized_value * (2**INTEGER - 1)
#         aux.append(scaled_value)
#     df_train[column_name]=aux
    
# display(df_train)

### NN training

In [8]:
# Begin NN training
NUMBER_OF_ATTRS = df_train.shape[1]

# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Normalize the data
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(df_train)

label_train = np.asarray(label_train).astype(np.float32)
#normalizer.adapt(numeric_features_test)

# Execute Trainning 
print('Starting training for',number_of_attributes,'attributes')
model = get_basic_model()
model.fit(df_train, label_train, epochs=EPOCHS, verbose=2, batch_size=BATCH_SIZE)

# and Testing
test_loss, one_test_acc =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
tf_predictions_probabilities = model.predict(df_test)

#Crate Confusion Matrix for better understanding of results
tf_predictions = []
for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)
  j_max = x.argmax()
  tf_predictions.append(j_max)

conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
print(conf_m)

Starting training for 3 attributes
Epoch 1/500
1/1 - 1s - 602ms/step - accuracy: 0.1899 - loss: 1.2278
Epoch 2/500
1/1 - 0s - 18ms/step - accuracy: 0.1899 - loss: 1.2267
Epoch 3/500
1/1 - 0s - 17ms/step - accuracy: 0.1899 - loss: 1.2256
Epoch 4/500
1/1 - 0s - 18ms/step - accuracy: 0.1899 - loss: 1.2245
Epoch 5/500
1/1 - 0s - 16ms/step - accuracy: 0.1899 - loss: 1.2234
Epoch 6/500
1/1 - 0s - 15ms/step - accuracy: 0.1350 - loss: 1.2223
Epoch 7/500
1/1 - 0s - 15ms/step - accuracy: 0.1350 - loss: 1.2213
Epoch 8/500
1/1 - 0s - 16ms/step - accuracy: 0.1350 - loss: 1.2202
Epoch 9/500
1/1 - 0s - 15ms/step - accuracy: 0.1350 - loss: 1.2192
Epoch 10/500
1/1 - 0s - 16ms/step - accuracy: 0.1350 - loss: 1.2181
Epoch 11/500
1/1 - 0s - 16ms/step - accuracy: 0.1350 - loss: 1.2171
Epoch 12/500
1/1 - 0s - 17ms/step - accuracy: 0.1730 - loss: 1.2160
Epoch 13/500
1/1 - 0s - 16ms/step - accuracy: 0.1730 - loss: 1.2149
Epoch 14/500
1/1 - 0s - 16ms/step - accuracy: 0.1730 - loss: 1.2138
Epoch 15/500
1/1 - 0s

In [9]:
# Model info
model.summary()
norm_layer_weights = model.layers[0].get_weights()[0] #weight
norm_layer_biases  = model.layers[0].get_weights()[1] #bias
hidden_layer_weights = model.layers[1].get_weights()[0] #weight
hidden_layer_biases  = model.layers[1].get_weights()[1] #bias
out_layer_weights = model.layers[2].get_weights()[0] #weight
out_layer_biases  = model.layers[2].get_weights()[1] #bias
print('\nInput -> Normalization Layer bias:\n',norm_layer_biases)
print('\nInput -> Normalization Layer weights:\n',norm_layer_weights)
print('\nNormalization -> Hidden Layer bias:\n',hidden_layer_biases)
print('\nNormalization -> Hidden Layer weights:\n',hidden_layer_weights)
print('\nHidden -> Output Layer bias:\n',out_layer_biases)
print('\nHidden -> Output Layer weights:\n',out_layer_weights,'\n')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ normalization (Normalization)        │ (None, 3)                   │               7 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │              12 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │              12 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 81 (332.00 B)

 Trainable params: 24 (96.00 B)

 Non-trainable params: 7 (32.00 B)

 Optimizer params: 50 (204.00 B)


Input -> Normalization Layer bias:
 [0.6153218  0.56095093 0.51726073]

Input -> Normalization Layer weights:
 [-0.14767933 -0.1308017  -0.21940929]

Normalization -> Hidden Layer bias:
 [-0.2804896  -0.08071144  0.01189113]

Normalization -> Hidden Layer weights:
 [[ 0.38790786 -0.08814368 -0.41493565]
 [-0.26708868  0.71702576  0.35362825]
 [-0.27892858 -0.09141283 -0.86742264]]

Hidden -> Output Layer bias:
 [-0.26221707  0.27602214 -0.39916393]

Hidden -> Output Layer weights:
 [[ 0.6986372   0.67080164 -0.5358617 ]
 [-0.40344965 -1.1579047   0.33075055]
 [-0.01508353 -0.17391126 -0.12919842]] 



In [24]:
print(df_test)


[[ 1.  1.  0.]
 [-1. -1. -1.]
 [-1. -1. -1.]
 [ 1. -1.  1.]
 [ 0.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 1. -1.  1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]
 [-1.  1. -1.]
 [-1. -1. -1.]
 [ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1. -1.  1.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]
 [-1. -1. -1.]
 [ 1.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]
 [-1. -1. -1.]
 [-1.  0. -1.]
 [-1.  0. -1.]
 [ 0.  1.  0.]
 [-1. -1. -1.]
 [ 0.  0.  0.]
 [-1.  0. -1.]
 [-1. -1. -1.]
 [-1. -1. -1.]
 [ 1.  0.  0.]
 [-1.  0. -1.]
 [-1. -1. -1.]
 [ 1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]
 [-1.  1. -1.]
 [ 1.  1.  1.]
 [-1.  1. -1.]
 [ 1.  0.  1.]
 [-1. -1. -1.]
 [-1.  1. -1.]
 [ 0. -1.  0.]
 [ 1.  0.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [-1.  0. -1.]
 [ 0.  0.  0.]
 [-1.  1. -1.]
 [ 1.  1.  1.]
 [ 1.  0.  0.]
 [-1.  1. -1.]
 [ 0.  0.  1.]
 [-1.  0. -1.]
 [ 0. -1.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]


In [26]:
# LUT table implementation: Internal Value Simulator

input_data = [-1, 0, 1] # all n possibilities a the unit type can measure
norm_data = np.zeros(input_data.shape)
h1_data = np.zeros(input_data.shape)
out_data = np.zeros(input_data.shape)

# manual norm layer implementation
for row in range(input_data.shape[0]):
    for column in range(input_data.shape[1]):
        #print(input_data[row][column])
        norm_data[row][column] = (input_data[row][column] * norm_layer_weights[column]) / norm_layer_biases[column]
print(norm_data)

# manual h1 layer implementation
for row in range(norm_data.shape[0]):
    for column in range(norm_data.shape[1]):
        #print(input_data[row][column])
        h1_data[row][column] = 






[[-0.24000341 -0.2331785  -0.        ]
 [ 0.24000341  0.2331785   0.42417541]
 [ 0.24000341  0.2331785   0.42417541]
 [-0.24000341  0.2331785  -0.42417541]
 [-0.         -0.         -0.        ]
 [-0.24000341 -0.         -0.        ]
 [-0.         -0.         -0.        ]
 [-0.24000341  0.2331785  -0.42417541]
 [-0.          0.2331785  -0.        ]
 [-0.         -0.         -0.        ]
 [ 0.24000341 -0.2331785   0.42417541]
 [ 0.24000341  0.2331785   0.42417541]
 [-0.24000341 -0.         -0.        ]
 [-0.          0.2331785  -0.        ]
 [-0.         -0.         -0.        ]
 [-0.         -0.         -0.        ]
 [-0.24000341  0.2331785  -0.42417541]
 [-0.          0.2331785  -0.        ]
 [-0.         -0.         -0.42417541]
 [ 0.24000341  0.2331785   0.42417541]
 [-0.24000341 -0.         -0.42417541]
 [-0.         -0.         -0.42417541]
 [-0.         -0.         -0.        ]
 [ 0.24000341  0.2331785   0.42417541]
 [ 0.24000341 -0.          0.42417541]
 [ 0.24000341 -0.        

In [10]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "number_of_attributes": number_of_attributes,
    "hidden_layer_nodes": hidden_layer_nodes,
    "output_layer_nodes": output_layer_nodes,
    "accuracy_test": one_test_acc,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS
    #"number_of_samples": number_of_samples,
    #"df_number": df_number
}

25-11-2024-14-41-59


In [11]:
#TO DO: use save_models instead of save_weights

# Model parameters json file generation, create file with date-time string to prevent unwated/accidental overwrites
os.chdir('../tf-params-reports/')
print(os.getcwd())

title_parameters_save = f"nn-nprint-{use_case}-model-parameters-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-{use_case}-model-parameters-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

# Model file with weights and other params
os.chdir('../tf-models/') 
print(os.getcwd())

title_model_save = f'nn-nprint-{use_case}-model-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.keras'
model.save(title_model_save)

# Model weights file generation
# os.chdir('../tf-model-weights/')
# print(os.getcwd())

# title_model_save = f'nn-nprint-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.weights.h5'
# model.save_weights(title_model_save) # Calling `save('my_model')` creates a SavedModel folder `my_model`.

# title_model_save = f'nn-nprint-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.weights.h5'
# model.save_weights(title_model_save) # Calling `save('my_model')` creates a SavedModel folder `my_model`.
    
# model.load_weights(f'nn-nprint-app-iden-model-weights-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.weights.h5')

C:\NN-P4\tf-params-reports
C:\NN-P4\tf-models


In [12]:
# Save timestamp
end = time.time()
print(end - start)

15.802741289138794


In [13]:

# Acervo - comandos úteis os python
# %pwd
# os.chdir(C:\NN-P4\nn-reports)
# cwd = os.getcwd()
# print(cwd)
# os.listdir()
# os.chdir('../nn-reports/')

In [14]:
# df_train = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/X.pkl')
# df_test = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/X_val.pkl')
# label_train = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/y.pkl')
# label_test = pd.read_pickle('/content/drive/MyDrive/nprint/nn-reproduction/app-iden/y_val.pkl')